# Pediatric DS AML vs TAM

Sleuth Analysis with Kallisto Quantitation Input

## Following the Pachterlab walkthrough
[Using p-value aggregation to obtain gene differential expression in datasets with multiple experimental conditions](
https://pachterlab.github.io/sleuth_walkthroughs/pval_agg/analysis.html)




### Introduction

Applying the same techniques used in the walk through - apply it to our own AML vs TAM

We need to set up our working directory.
And we need 

* `cowplot` - for making prettier plots and plots with grids.  and 
* `biomaRt` - for extracting the Ensembl transcript to gene mapping



In [2]:
install.packages("cowplot")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [3]:
library (cowplot)

In [6]:
if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(version = "3.14")

Bioconductor version '3.14' is out-of-date; the current release version '3.15'
  is available with R version '4.2'; see https://bioconductor.org/install

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.1 (2021-08-10)

Installing package(s) 'BiocVersion'

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'BiocGenerics', 'blob', 'brew', 'brio', 'broom',
  'bslib', 'callr', 'caret', 'class', 'cli', 'clipr', 'colorspace',
  'commonmark', 'conflicted', 'covr', 'cpp11', 'crayon', 'credentials',
  'crosstalk', 'curl', 'data.table', 'DBI', 'dbplyr', 'desc', 'devtools',
  'dials', 'diffobj', 'digest', 'dplyr', 'DT', 'dtplyr', 'e1071', 'evaluate',
  'fansi', 'farver', 'forcats', 'foreach', 'forecast', 'fs', 'furrr', 'future',
  'future.apply', 'gargle', 'generics', 'gert

In [7]:
BiocManager::install("biomaRt")

'getOption("repos")' replaces Bioconductor standard repositories, see
'?repositories' for details

replacement repositories:
    CRAN: https://cran.r-project.org


Bioconductor version 3.14 (BiocManager 1.30.18), R 4.1.1 (2021-08-10)

Installing package(s) 'biomaRt'

also installing the dependencies ‘zlibbioc’, ‘GenomeInfoDbData’, ‘XVector’, ‘GenomeInfoDb’, ‘BiocGenerics’, ‘png’, ‘Biostrings’, ‘Biobase’, ‘IRanges’, ‘KEGGREST’, ‘filelock’, ‘XML’, ‘AnnotationDbi’, ‘BiocFileCache’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done

Old packages: 'backports', 'blob', 'brew', 'brio', 'broom', 'bslib', 'callr',
  'caret', 'class', 'cli', 'clipr', 'colorspace', 'commonmark', 'conflicted',
  'covr', 'cpp11', 'crayon', 'credentials', 'crosstalk', 'curl', 'data.table',
  'DBI', 'dbplyr', 'desc', 'devtools', 'dials', 'diffobj', 'digest', 'dplyr',
  'DT', 'dtplyr', 'e1071', 'evaluate', 'fansi', 'farver', 'forcats', 'foreach',
  'forecast', 'fs', 'furrr', 'future', 'f